In [ ]:
# ===============================
# PART 1 — Imports & Config
# ===============================

import pandas as pd
import numpy as np
import requests
from sklearn.preprocessing import StandardScaler

TRAFFIC_PATH = "/content/drive/MyDrive/trafficflow/PEMS-BAY.csv"
SAVE_DIR = "/content/drive/MyDrive/trafficflow/"


In [ ]:
# ===============================
# PART 2 — Load Traffic
# ===============================

traffic = pd.read_csv(TRAFFIC_PATH)

traffic.rename(columns={"Unnamed: 0": "timestamp"}, inplace=True)
traffic["timestamp"] = pd.to_datetime(traffic["timestamp"])
traffic.set_index("timestamp", inplace=True)

print("Traffic:", traffic.shape)
traffic.head()


Traffic: (52116, 325)


,400001,400017,400030,400040,400045,400052,400057,400059,400065,400069,...,409525,409526,409528,409529,413026,413845,413877,413878,414284,414694
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,69.0,...,68.8,67.9,68.8,68.0,69.2,68.9,70.4,68.8,71.1,68.0
2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,68.2,...,68.4,67.3,68.4,67.6,70.4,68.8,70.1,68.4,70.8,67.4
2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,67.8,...,68.4,67.4,68.4,67.5,70.2,68.3,69.8,68.4,70.5,67.9
2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,67.8,...,68.5,67.5,68.5,67.5,70.4,68.7,70.2,68.4,70.8,67.6
2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,67.8,...,68.5,67.7,68.5,67.4,69.6,69.1,70.0,68.4,71.0,67.9


In [ ]:
# ===============================
# PART 3 — Download Weather
# ===============================

url = (
    "https://archive-api.open-meteo.com/v1/archive?"
    "latitude=37.77&longitude=-122.42"
    "&start_date=2017-01-01"
    "&end_date=2017-05-31"
    "&hourly=temperature_2m,rain,wind_speed_10m"
    "&timezone=UTC"
)

weather_json = requests.get(url).json()
hourly = weather_json["hourly"]

weather = pd.DataFrame({
    "timestamp": hourly["time"],
    "temp": hourly["temperature_2m"],
    "rain": hourly["rain"],
    "wind": hourly["wind_speed_10m"]
})

weather["timestamp"] = pd.to_datetime(weather["timestamp"])
weather.set_index("timestamp", inplace=True)

weather.head()


,temp,rain,wind
timestamp,,,
2017-01-01 00:00:00,10.2,0.0,10.0
2017-01-01 01:00:00,8.8,0.0,10.6
2017-01-01 02:00:00,8.9,0.0,16.1
2017-01-01 03:00:00,9.0,0.0,17.3
2017-01-01 04:00:00,8.9,0.0,17.7


In [ ]:
# ===============================
# PART 4 — Resample Weather
# ===============================

weather_5min = weather.resample("5min").ffill()

print("Weather:", weather_5min.shape)
weather_5min.head()


Weather: (43477, 3)


,temp,rain,wind
timestamp,,,
2017-01-01 00:00:00,10.2,0.0,10.0
2017-01-01 00:05:00,10.2,0.0,10.0
2017-01-01 00:10:00,10.2,0.0,10.0
2017-01-01 00:15:00,10.2,0.0,10.0
2017-01-01 00:20:00,10.2,0.0,10.0


In [ ]:
# ===============================
# PART 5 — Merge
# ===============================

data = traffic.join(weather_5min)

print("Merged:", data.shape)
data.head()


Merged: (52116, 328)


,400001,400017,400030,400040,400045,400052,400057,400059,400065,400069,...,409529,413026,413845,413877,413878,414284,414694,temp,rain,wind
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,69.0,...,68.0,69.2,68.9,70.4,68.8,71.1,68.0,10.2,0.0,10.0
2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,68.2,...,67.6,70.4,68.8,70.1,68.4,70.8,67.4,10.2,0.0,10.0
2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,67.8,...,67.5,70.2,68.3,69.8,68.4,70.5,67.9,10.2,0.0,10.0
2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,67.8,...,67.5,70.4,68.7,70.2,68.4,70.8,67.6,10.2,0.0,10.0
2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,67.8,...,67.4,69.6,69.1,70.0,68.4,71.0,67.9,10.2,0.0,10.0


In [ ]:
# =========================================
# PART — Holidays (FINAL CLEAN VERSION)
# =========================================

import requests
import pandas as pd

# 1️⃣ Download US public holidays for 2017
holiday_json = requests.get(
    "https://date.nager.at/api/v3/PublicHolidays/2017/US"
).json()

holiday_dates = pd.to_datetime([h["date"] for h in holiday_json])


# 2️⃣ Keep only dates inside your traffic period
start, end = data.index.min(), data.index.max()

holiday_dates = holiday_dates[
    (holiday_dates >= start) &
    (holiday_dates <= end)
]


# 3️⃣ Convert both sides to midnight timestamps (IMPORTANT)
holiday_midnight = holiday_dates.normalize()


# 4️⃣ Create holiday column (SAFE MATCHING)
data["holiday"] = data.index.normalize().isin(holiday_midnight).astype(int)


# 5️⃣ Weekend column
data["weekend"] = (data.index.dayofweek >= 5).astype(int)


# 6️⃣ Check
data[["holiday", "weekend"]].head(10)


,holiday,weekend
timestamp,,
2017-01-01 00:00:00,0,1
2017-01-01 00:05:00,0,1
2017-01-01 00:10:00,0,1
2017-01-01 00:15:00,0,1
2017-01-01 00:20:00,0,1
2017-01-01 00:25:00,0,1
2017-01-01 00:30:00,0,1
2017-01-01 00:35:00,0,1
2017-01-01 00:40:00,0,1


In [ ]:
data.loc["2017-01-02 00:00:00":"2017-01-02 00:20:00", ["holiday","weekend"]]


,holiday,weekend
timestamp,,
2017-01-02 00:00:00,1,0
2017-01-02 00:05:00,1,0
2017-01-02 00:10:00,1,0
2017-01-02 00:15:00,1,0
2017-01-02 00:20:00,1,0


In [ ]:
# =========================================
# ADD HOLIDAY + WEEKEND TO MAIN DATA
# =========================================

import requests
import pandas as pd

# 1️⃣ download US holidays
holiday_json = requests.get(
    "https://date.nager.at/api/v3/PublicHolidays/2017/US"
).json()

holiday_dates = pd.to_datetime([h["date"] for h in holiday_json])


# 2️⃣ filter only your dataset range
start, end = data.index.min(), data.index.max()

holiday_dates = holiday_dates[
    (holiday_dates >= start) &
    (holiday_dates <= end)
]


# 3️⃣ normalize for safe matching
holiday_midnight = holiday_dates.normalize()


# 4️⃣ add columns to MAIN dataset
data["holiday"] = data.index.normalize().isin(holiday_midnight).astype(int)

data["weekend"] = (data.index.dayofweek >= 5).astype(int)


# 5️⃣ verify
print("Final shape:", data.shape)
data[["holiday","weekend"]].head(10)


Final shape: (52116, 330)


,holiday,weekend
timestamp,,
2017-01-01 00:00:00,0,1
2017-01-01 00:05:00,0,1
2017-01-01 00:10:00,0,1
2017-01-01 00:15:00,0,1
2017-01-01 00:20:00,0,1
2017-01-01 00:25:00,0,1
2017-01-01 00:30:00,0,1
2017-01-01 00:35:00,0,1
2017-01-01 00:40:00,0,1


In [ ]:
print("Shape:", data.shape)
print("\nColumns:\n")
print(list(data.columns))


Shape: (52116, 330)

Columns:

['400001', '400017', '400030', '400040', '400045', '400052', '400057', '400059', '400065', '400069', '400073', '400084', '400085', '400088', '400096', '400097', '400100', '400104', '400109', '400122', '400147', '400148', '400149', '400158', '400160', '400168', '400172', '400174', '400178', '400185', '400201', '400206', '400209', '400213', '400221', '400222', '400227', '400236', '400238', '400240', '400246', '400253', '400257', '400258', '400268', '400274', '400278', '400280', '400292', '400296', '400298', '400330', '400336', '400343', '400353', '400372', '400394', '400400', '400414', '400418', '400429', '400435', '400436', '400440', '400449', '400457', '400461', '400464', '400479', '400485', '400499', '400507', '400508', '400514', '400519', '400528', '400545', '400560', '400563', '400567', '400581', '400582', '400586', '400637', '400643', '400648', '400649', '400654', '400664', '400665', '400668', '400673', '400677', '400687', '400688', '400690', '400700'

In [ ]:
print(type(data.index))
print(data.index[:5])


<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 00:05:00',
               '2017-01-01 00:10:00', '2017-01-01 00:15:00',
               '2017-01-01 00:20:00'],
              dtype='datetime64[ns]', name='timestamp', freq=None)


In [ ]:
data[["temp","rain","wind"]].head()


,temp,rain,wind
timestamp,,,
2017-01-01 00:00:00,10.2,0.0,10.0
2017-01-01 00:05:00,10.2,0.0,10.0
2017-01-01 00:10:00,10.2,0.0,10.0
2017-01-01 00:15:00,10.2,0.0,10.0
2017-01-01 00:20:00,10.2,0.0,10.0


In [ ]:
data["holiday"].value_counts()


,count
holiday,
0,50100
1,2016


In [ ]:
data["weekend"].value_counts()


,count
weekend,
0,37440
1,14676


In [ ]:
sensor_cols = [c for c in data.columns if c.isdigit()]
print("Sensors:", len(sensor_cols))


Sensors: 325


In [ ]:
data_reset = data.reset_index()
data_reset.head()


,timestamp,400001,400017,400030,400040,400045,400052,400057,400059,400065,...,413845,413877,413878,414284,414694,temp,rain,wind,holiday,weekend
0,2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,...,68.9,70.4,68.8,71.1,68.0,10.2,0.0,10.0,0,1
1,2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,...,68.8,70.1,68.4,70.8,67.4,10.2,0.0,10.0,0,1
2,2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,...,68.3,69.8,68.4,70.5,67.9,10.2,0.0,10.0,0,1
3,2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,...,68.7,70.2,68.4,70.8,67.6,10.2,0.0,10.0,0,1
4,2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,...,69.1,70.0,68.4,71.0,67.9,10.2,0.0,10.0,0,1


In [ ]:
display(data.head())

,400001,400017,400030,400040,400045,400052,400057,400059,400065,400069,...,413845,413877,413878,414284,414694,temp,rain,wind,holiday,weekend
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,69.0,...,68.9,70.4,68.8,71.1,68.0,10.2,0.0,10.0,0,1
2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,68.2,...,68.8,70.1,68.4,70.8,67.4,10.2,0.0,10.0,0,1
2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,67.8,...,68.3,69.8,68.4,70.5,67.9,10.2,0.0,10.0,0,1
2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,67.8,...,68.7,70.2,68.4,70.8,67.6,10.2,0.0,10.0,0,1
2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,67.8,...,69.1,70.0,68.4,71.0,67.9,10.2,0.0,10.0,0,1


In [ ]:
from google.colab import files

# convert index → column so timestamp is saved
final_df = data.reset_index()

file_name = "pems_bay_final_processed_dataset.csv"

final_df.to_csv(file_name, index=False)

files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =========================================
# LOAD SAVED DATASET FROM DRIVE
# =========================================

import pandas as pd
import numpy as np
from google.colab import files

file_path = "/content/drive/MyDrive/trafficflow/pems_bay_final_processed_dataset.csv"

data = pd.read_csv(file_path, parse_dates=["timestamp"], index_col="timestamp")

print("Loaded:", data.shape)


# =========================================
# ADD NEW FEATURES (TIME + ROLLING)
# =========================================

# find sensor columns automatically (all numeric IDs)
sensor_cols = [c for c in data.columns if c.isdigit()]


# ---------- TIME FEATURES ----------
data["hour"] = data.index.hour
data["dayofweek"] = data.index.dayofweek

data["hour_sin"] = np.sin(2*np.pi*data["hour"]/24)
data["hour_cos"] = np.cos(2*np.pi*data["hour"]/24)

data["dow_sin"] = np.sin(2*np.pi*data["dayofweek"]/7)
data["dow_cos"] = np.cos(2*np.pi*data["dayofweek"]/7)


# ---------- ROLLING TRAFFIC TRENDS ----------
data["mean_15min"] = data[sensor_cols].mean(axis=1).rolling(3).mean()
data["mean_30min"] = data[sensor_cols].mean(axis=1).rolling(6).mean()


# fill edges created by rolling
data = data.ffill().bfill()

print("After new features:", data.shape)


# =========================================
# SAVE UPDATED DATASET
# =========================================

new_path = "/content/drive/MyDrive/trafficflow/pems_bay_final_with_extra_features.csv"

data.reset_index().to_csv(new_path, index=False)

print("Saved:", new_path)


# =========================================
# DOWNLOAD TO PC
# =========================================

files.download(new_path)


Loaded: (52116, 330)
After new features: (52116, 338)
Saved: /content/drive/MyDrive/trafficflow/pems_bay_final_with_extra_features.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data.shape

(52116, 338)

In [ ]:
data.head()

,400001,400017,400030,400040,400045,400052,400057,400059,400065,400069,...,holiday,weekend,hour,dayofweek,hour_sin,hour_cos,dow_sin,dow_cos,mean_15min,mean_30min
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,71.4,67.8,70.5,67.4,68.8,66.6,66.8,68.0,66.8,69.0,...,0,1,0,6,0.0,1.0,-0.781831,0.62349,68.282256,68.194205
2017-01-01 00:05:00,71.6,67.5,70.6,67.5,68.7,66.6,66.8,67.8,66.5,68.2,...,0,1,0,6,0.0,1.0,-0.781831,0.62349,68.282256,68.194205
2017-01-01 00:10:00,71.6,67.6,70.2,67.4,68.7,66.1,66.8,67.8,66.2,67.8,...,0,1,0,6,0.0,1.0,-0.781831,0.62349,68.282256,68.194205
2017-01-01 00:15:00,71.1,67.5,70.3,68.0,68.5,66.7,66.6,67.7,65.9,67.8,...,0,1,0,6,0.0,1.0,-0.781831,0.62349,68.170564,68.194205
2017-01-01 00:20:00,71.7,67.8,70.2,68.1,68.4,66.9,66.1,67.7,66.1,67.8,...,0,1,0,6,0.0,1.0,-0.781831,0.62349,68.134769,68.194205
